In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
abdoashraf90_ahqad_arabic_healthcare_q_and_a_dataset_path = kagglehub.dataset_download('abdoashraf90/ahqad-arabic-healthcare-q-and-a-dataset')

print('Data source import complete.')


Data source import complete.


In [2]:
pip install farasapy

In [3]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from farasa.stemmer import FarasaStemmer
from collections import defaultdict
# from preprocessing import tokenize_arabic, remove_stopwords, clean_arabic_text
import tkinter as tk
from tkinter import ttk
# from retrieval import preprocess_query, retrieve_documents
# from ranking import rank_documents
# from indexing import build_inverted_index, build_document_index
# from preprocessing import load_data, preprocess_documents

# Download Arabic resources
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
print("Downloading dataset from Kaggle...")
import kagglehub
import pandas as pd
import os

# Download dataset
dataset_path = kagglehub.dataset_download('abdoashraf90/ahqad-arabic-healthcare-q-and-a-dataset')

# Correct path handling (Windows/Linux compatible)
dataset_file = os.path.join(dataset_path, "AHQAD.csv")

# Read CSV (single argument - either file path or file object)
df = pd.read_csv(dataset_file)  # Removed the second argument
df['id'] = df.index

In [5]:
df.head()

,Unnamed: 0,Question,Answer,Category,id
0,0,اجريت صورة رنين مغناطيسي للثدي ظهر منطقة تعزيز...,المهم بالموضوع ما هي درجة ال BIRADS الموصوفة ل...,جراحة عامة,0
1,1,انا اعاني من اشياء تجبرني على فعل امور لا اريد...,الف سلامه عليك ، انت في حاجه الي معالج نفسي لم...,أمراض نفسية,1
2,2,أريد الاستفسار عن مضاعفات عملية استئصال ورم ال...,كغيرها من العمليات الجراحية ترتبط عملية استئصا...,أمراض الغدد الصماء,2
3,3,اليوم حاسس ان عندى حاجه فى الدبر من جوا مش عاي...,يغلب ان تعكس الحالة بواسير شرجية داخلية من الد...,جراحة عامة,3
4,4,ليه فجأة بحس انى مش عارفة اتنفس وجسمى بينمل وي...,من المحتمل عضوي او تاثير جانبي من اي ادويه بيت...,أمراض نفسية,4


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 808472 entries, 0 to 808471
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  808472 non-null  int64 
 1   Question    808471 non-null  object
 2   Answer      808472 non-null  object
 3   Category    808472 non-null  object
 4   id          808472 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 30.8+ MB


In [7]:
import numpy as np

# Randomly set 5% of 'column_name' to NaN
df.loc[df.sample(frac=0.50).index, 'Question'] = np.nan
df.loc[df.sample(frac=0.50).index, 'Answer'] = np.nan



In [8]:
df.isnull().sum()

,0
Unnamed: 0,0
Question,404237
Answer,404236
Category,0
id,0


In [9]:
df.dropna(inplace=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 201713 entries, 1 to 808471
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  201713 non-null  int64 
 1   Question    201713 non-null  object
 2   Answer      201713 non-null  object
 3   Category    201713 non-null  object
 4   id          201713 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 9.2+ MB


In [11]:
df.isnull().sum()

,0
Unnamed: 0,0
Question,0
Answer,0
Category,0
id,0


# PreProcessing

In [12]:
import re
from nltk.stem.isri import ISRIStemmer
from nltk.corpus import stopwords
import unicodedata

def clean_arabic_text(text):
    """Clean Arabic text with more comprehensive preprocessing"""
    # Normalize Arabic characters (e.g., convert آ to ا)
    text = unicodedata.normalize('NFKD', text)

    # Remove diacritics (harakat)
    text = re.sub(r'[\u064B-\u065F\u0670]', '', text)

    # Remove punctuation and special characters (keep Arabic and numbers)
    text = re.sub(r'[^\w\u0600-\u06FF\s]', ' ', text)

    # Remove English letters if present
    text = re.sub(r'[a-zA-Z]', '', text)

    # Remove repeated characters (more than 2 repetitions)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)

    # Remove extra spaces and line breaks
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def regex_tokenize_arabic(text):
    """Enhanced tokenization for Arabic text"""
    # Tokenize words and keep numbers
    tokens = re.findall(r'[\u0600-\u06FF0-9]+', text)
    return tokens

def normalize_arabic_text(text):
    """Normalize Arabic text variations"""
    # Common replacements
    replacements = {
        'ة': 'ه',  # Convert taa marbuta to haa
        'إ': 'ا',  # Convert alif with hamza below to alif
        'أ': 'ا',  # Convert alif with hamza above to alif
        'آ': 'ا',  # Convert alif with madda to alif
        'ى': 'ي',  # Convert alif maksura to yaa
    }
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text

def tokenize_arabic(text):
    """Tokenize, normalize and stem Arabic text"""
    stemmer = ISRIStemmer()
    tokens = regex_tokenize_arabic(text)

    # Normalize tokens
    normalized_tokens = [normalize_arabic_text(token) for token in tokens]

    # Stem tokens
    stemmed_tokens = [stemmer.stem(token) for token in normalized_tokens]

    return stemmed_tokens

def remove_stopwords(tokens):
    """Remove Arabic stopwords with additional custom stopwords"""
    arabic_stopwords = set(stopwords.words('arabic'))
    # Add common modern Arabic stopwords not in NLTK
    custom_stopwords = {
        'هذا', 'هذه', 'ذلك', 'هؤلاء', 'كان', 'يكون', 'قال', 'يكون',
        'قد', 'لا', 'ما', 'إن', 'أن', 'حتى', 'إلى', 'على', 'في'
    }
    arabic_stopwords.update(custom_stopwords)
    return [token for token in tokens if token not in arabic_stopwords]

def preprocess_documents(df):
    """Enhanced preprocessing for all documents in the dataframe"""
    processed_docs = []
    for _, row in df.iterrows():
        text = f"{row['Question']} {row['Answer']}"

        # Clean text
        cleaned = clean_arabic_text(text)

        # Tokenize, normalize and stem
        tokens = tokenize_arabic(cleaned)

        # Remove stopwords
        filtered = remove_stopwords(tokens)

        # Remove single character tokens (except numbers)
        filtered = [token for token in filtered if len(token) > 1 or token.isdigit()]

        processed_docs.append({
            'id': row['id'],
            'original_text': text,
            'processed_text': ' '.join(filtered),
            'category': row['Category']
        })

    return processed_docs

In [13]:
processed_docs = preprocess_documents(df)


In [14]:
processed_docs[0]

{'id': 1,
 'original_text': 'انا اعاني من اشياء تجبرني على فعل امور لا اريدها واذا لم اعطها اهتمام تزيد وياتي تورتر شديد مع ضيق وصور في الذهن محرمه فالدين وتزعجني كثيراً اعاني سنه احتاج... الف سلامه عليك ، انت في حاجه الي معالج نفسي لمساعدتك',
 'processed_text': 'انا اعا شيء جبر علي فعل امر ارد وذا اعط همم تزد ويت رتر شدد ضيق وصر ذهن حرم فلد زعج كثر اعا سنه حاج الف سلم علك انت حجه الي علج لمساعدتك',
 'category': 'أمراض نفسية'}

# Indexing


In [33]:
from collections import defaultdict,Counter
import logging

def build_inverted_index(documents):
    """Build inverted index from processed documents with validation"""
    inverted_index = defaultdict(list)

    if not documents:
        logging.warning("Empty documents list provided to build_inverted_index")
        return inverted_index

    valid_docs = 0
    empty_docs = 0

    for doc in documents:
        if not isinstance(doc, dict) or 'id' not in doc or 'processed_text' not in doc:
            logging.warning(f"Invalid document format: {doc}")
            continue

        doc_id = doc['id']
        processed_text = doc['processed_text']

        if not processed_text or not isinstance(processed_text, str):
            empty_docs += 1
            continue

        terms = processed_text.split()
        if not terms:
            empty_docs += 1
            continue

        term_freq = defaultdict(int)
        for term in terms:
            if term:  # Skip empty strings
                term_freq[term] += 1

        for term, freq in term_freq.items():
            inverted_index[term].append((doc_id, freq))

        valid_docs += 1

    logging.info(
        f"Inverted index built with {valid_docs} docs. "
        f"Skipped {empty_docs} empty documents."
    )
    return inverted_index

def build_document_index(documents):
    """Build document index with robust validation"""
    doc_index = {}

    if not documents:
        logging.warning("Empty documents list provided to build_document_index")
        return doc_index

    valid_docs = 0
    invalid_docs = 0

    for doc in documents:
        if not isinstance(doc, dict):
            invalid_docs += 1
            continue

        try:
            doc_id = doc['id']
            doc_index[doc_id] = {
                'original': doc.get('original_text', ''),
                'processed': doc.get('processed_text', ''),
                # 'category': doc.get('category', None)
                'category': doc.get('category', 'Uncategorized')
            }
            valid_docs += 1
        except KeyError as e:
            logging.warning(f"Missing key in document {doc}: {e}")
            invalid_docs += 1

    logging.info(
        f"Document index built with {valid_docs} docs. "
        f"Skipped {invalid_docs} invalid documents."
    )
    return doc_index

In [34]:
inverted_index = build_inverted_index(processed_docs)
doc_index = build_document_index(processed_docs)
inverted_index

In [36]:
for term, postings in inverted_index.items():
    print(f"Term: {term}")
    print(f"postings: {postings}")

    for doc_id, freq in postings:
        print(f"  Document ID: {doc_id}, Frequency: {freq}")
    break


Streaming output truncated to the last 5000 lines.
  Document ID: 770782, Frequency: 2
  Document ID: 770783, Frequency: 6
  Document ID: 770797, Frequency: 12
  Document ID: 770802, Frequency: 6
  Document ID: 770803, Frequency: 2
  Document ID: 770806, Frequency: 2
  Document ID: 770817, Frequency: 4
  Document ID: 770819, Frequency: 2
  Document ID: 770822, Frequency: 4
  Document ID: 770828, Frequency: 4
  Document ID: 770830, Frequency: 2
  Document ID: 770831, Frequency: 4
  Document ID: 770833, Frequency: 2
  Document ID: 770855, Frequency: 8
  Document ID: 770859, Frequency: 4
  Document ID: 770868, Frequency: 6
  Document ID: 770890, Frequency: 2
  Document ID: 770891, Frequency: 4
  Document ID: 770892, Frequency: 4
  Document ID: 770899, Frequency: 2
  Document ID: 770904, Frequency: 2
  Document ID: 770909, Frequency: 2
  Document ID: 770921, Frequency: 2
  Document ID: 770923, Frequency: 70
  Document ID: 770953, Frequency: 6
  Document ID: 770958, Frequency: 2
  Document 

# Retrieval

In [18]:
# Search and Ranking Functions
def preprocess_query(query):
     cleaned = clean_arabic_text(query)

        # Tokenize, normalize and stem
     tokens = tokenize_arabic(cleaned)

        # Remove stopwords
     filtered = remove_stopwords(tokens)

        # Remove single character tokens (except numbers)
     filtered = [token for token in filtered if len(token) > 1 or token.isdigit()]
     return filtered

def retrieve_documents(query_terms, inverted_index):
    """Retrieve documents containing query terms"""
    relevant_docs = set()
    for term in query_terms:
        if term in inverted_index:
            docs = [doc_id for doc_id, _ in inverted_index[term]]
            relevant_docs.update(docs)
    return list(relevant_docs)

def rank_documents(query_terms, relevant_docs, tfidf_index, doc_index):
    """Rank documents by cosine similarity"""
    doc_scores = defaultdict(float)
    query_vector = {term: 1 for term in query_terms}

    for doc_id in relevant_docs:
        doc_vector = defaultdict(float)
        doc_terms = doc_index[doc_id]['processed'].split()

        for term in set(doc_terms):
            if term in tfidf_index:
                for t_doc_id, score in tfidf_index[term]:
                    if t_doc_id == doc_id:
                        doc_vector[term] = score
                        break

        dot_product = sum(query_vector.get(term, 0) * doc_vector.get(term, 0))
        query_norm = math.sqrt(sum(score**2 for score in query_vector.values()))
        doc_norm = math.sqrt(sum(score**2 for score in doc_vector.values()))

        similarity = dot_product / (query_norm * doc_norm) if (query_norm * doc_norm) > 0 else 0
        doc_scores[doc_id] = similarity

    return sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)


# Ranking

In [19]:
import math
# Calculate TF-IDF scores
import math

def calculate_tfidf_scores(inverted_index, num_docs):
    """Calculate TF-IDF scores for all terms with safety checks"""
    tfidf_index = {}
    for term, doc_list in inverted_index.items():
        # Skip terms that don't appear in any documents
        if not doc_list or len(doc_list) == 0:
            continue

        # Calculate IDF with smoothing to avoid division by zero
        idf = math.log((num_docs + 1) / (len(doc_list) + 1)) + 1  # Additive smoothing

        # Calculate TF-IDF scores for each document
        tfidf_scores = [(doc_id, tf * idf) for doc_id, tf in doc_list]
        tfidf_index[term] = tfidf_scores

    return tfidf_index


def rank_documents(query_terms, relevant_docs, tfidf_index, doc_index):
    """Rank documents by cosine similarity"""
    doc_scores = defaultdict(float)

    # Calculate query vector (binary)
    query_vector = {term: 1 for term in query_terms}

    for doc_id in relevant_docs:
        doc_vector = defaultdict(float)

        # Get all terms in this document
        doc_terms = doc_index[doc_id]['processed'].split()

        # Build document vector with TF-IDF scores
        for term in set(doc_terms):
            if term in tfidf_index:
                # Find this doc's TF-IDF score for the term
                for t_doc_id, score in tfidf_index[term]:
                    if t_doc_id == doc_id:
                        doc_vector[term] = score
                        break

        # Calculate cosine similarity
        dot_product = sum(query_vector.get(term, 0) * doc_vector.get(term, 0)
                         for term in query_terms)
        query_norm = math.sqrt(sum(score**2 for score in query_vector.values()))
        doc_norm = math.sqrt(sum(score**2 for score in doc_vector.values()))

        if query_norm * doc_norm == 0:
            similarity = 0
        else:
            similarity = dot_product / (query_norm * doc_norm)

        doc_scores[doc_id] = similarity

    # Sort by score descending
    return sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)

In [20]:

doc_index = build_document_index(processed_docs)
num_docs = len(doc_index)

# Evaluation

In [31]:
# Evaluation Function
def evaluate_search(results, relevant_docs):
    """Calculate precision, recall, and F1 score with ID matching"""
    retrieved = set([str(doc_id) for doc_id, _ in results])
    relevant = set([str(doc_id) for doc_id in relevant_docs])

    true_positives = len(retrieved & relevant)
    false_positives = len(retrieved - relevant)
    false_negatives = len(relevant - retrieved)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

# Search Function with Evaluation
def search(query):
    if not query.strip():
        return "يرجى إدخال سؤال صحيح", "", ""

    # Preprocess and TF-IDF (assumes you have these functions/data)
    query_terms = preprocess_query(query)
    tfidf_index = calculate_tfidf_scores(inverted_index, num_docs)
    relevant_docs = retrieve_documents(query_terms, inverted_index)

    if not relevant_docs:
        return "لا توجد نتائج مطابقة لبحثك", "", ""

    # Rank retrieved docs
    ranked_results = rank_documents(query_terms, relevant_docs, tfidf_index, doc_index)

    # Evaluate top-5 retrieved results against all relevant docs
    precision, recall, f1 = evaluate_search(ranked_results[:5], relevant_docs)
    eval_text = f"الدقة: {precision:.2f} | الاستدعاء: {recall:.2f} | F1: {f1:.2f}"

    # Format results for display
    results_text = ""
    for i, (doc_id, score) in enumerate(ranked_results[:5], 1):
        original_text = doc_index[doc_id]['original']
        results_text += f"{i}. [التقييم: {score:.3f}]\n{original_text}\n\n"

    return results_text, eval_text, f"تم العثور على {len(ranked_results)} نتيجة"

In [22]:
# for term, postings in inverted_index.items():
#     print(f"Term: {term}")
#     for doc_id, freq in postings:
#         print(f"  Document ID: {doc_id}, Frequency: {freq}")
#     break

# GUI Interface

In [23]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 130.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.9 MB/s eta 0:00:00


In [32]:
search("ما هي اعراض ضغط الدم؟")

('1. [التقييم: 0.612]\nهل البيض يرفع الضغط أي ضغط الدم؟ لا يوجد رابط مباشر بينه وبين الضغط\n\n2. [التقييم: 0.453]\nكيف اعرف اني اعاني من مرض الضغط؟ وهل يوجد ارتفاع ضغط دم عابر؟ يمكن تشخيص ارتفاع ضغط الدم عن طريق إجراء قياسات دقيقة ومتكررة لضغط الدم من قبل أحد مقدمي الرعاية الصحية المختصين، وتدل قراءات ضغط الدم المختلفة على الآتي:\n\nضغط الدم الطبيعي: ضغط انقباضي أقل من 120، وضغط انبساطي أقل من 80.\nارتفاع ضغط الدم الطفيف: ضغط انقباضي من 120 إلى 129، وضغط انبساطي أقل من 80.\nالمرحلة الأولى من ارتفاع ضغط الدم: ضغط انقباضي من 130 إلى 139، وضغط انبساطي من 80 إلى 89.\nالمرحلة الثانية من ارتفاع ضغط الدم: ضغط انقباضي 140 أو أكثر، وضغط انبساطي 90 أو أكثر.\n\nإذا تم قياس ضغط دمك على ثلاث مرات منفصلة على الأقل، وكانت النتائج في كل مرة ضمن فئة ارتفاع ضغط الدم، فهذا يعني أنك تعاني من ارتفاع ضغط الدم.\n\xa0\nهل يوجد ارتفاع ضغط دم عابر؟\n\xa0\nنعم، يوجد ارتفاع ضغط دم عابر، يمكن أن يحدث ارتفاع ضغط الدم بسبب مجموعة متنوعة من العوامل، بما في ذلك:\n\nالقلق أو التوتر.\nتناول بعض الأدوية، مثل أدوية مسكنات

In [37]:
# Create Gradio Interface
import gradio as gr

with gr.Blocks(title="محرك بحث الرعاية الصحية باللغة العربية") as demo:
    gr.Markdown("# 🏥 محرك بحث الرعاية الصحية باللغة العربية")
    gr.Markdown("أدخل سؤالك الصحي باللغة العربية للحصول على إجابات مستخرجة من قاعدة بيانات طبية.")

    with gr.Row():
        query_input = gr.Textbox(label="🔍 بحث", placeholder="مثال: ما هي أعراض ضغط الدم؟")
        search_btn = gr.Button("ابحث")

    with gr.Row():
        results_output = gr.Textbox(label="النتائج", lines=15, interactive=False)

    with gr.Row():
        eval_output = gr.Textbox(label="📊 تقييم النتائج", interactive=False)
        count_output = gr.Textbox(label="📄 عدد النتائج", interactive=False)

    search_btn.click(
        fn=search,
        inputs=query_input,
        outputs=[results_output, eval_output, count_output]
    )

# Launch the demo
demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6901454dae60c9f4ad.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
